In [11]:
from models import get_model, avail_models, norm_vals
from video_dataset import get_data_loader
import torch
import gc
import torch.nn as nn
import torchvision.models as models
from torch.profiler import profile, ProfilerActivity, record_function

In [2]:
device = torch.device('cuda')

In [5]:
print("avail_models: ")
avail_m = avail_models()
print(', '.join(avail_m))

avail_models: 
S3D, R3D_18, R(2+1)D_18, Swin3D_T, Swin3D_S, Swin3D_B, MViTv2_S, MViTv1_B


## Input

In [ ]:
fs = 224 #frame size
nf = 16 #num frames

nvals = norm_vals(avail_m[0]) #normalisation won't make a difference in this case
# testloader = get_data_loader(
#     mean=nvals['mean'],
#     std=nvals['std'],
#     frame_size=fs,
#     num_frames=nf,
# )

torch.Size([2, 3, 16, 224, 224])


## First test on S3D

In [6]:
nc = 100 #num classes
dropout = 0.0 #no dropout
model = get_model(avail_m[0], nc, dropout)

In [7]:
print(model)

S3D_basic(
  (backbone): ModuleList(
    (0): Sequential(
      (0): TemporalSeparableConv(
        (0): Conv3dNormActivation(
          (0): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
          (1): BatchNorm3d(64, eps=0.001, momentum=0.001, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv3dNormActivation(
          (0): Conv3d(64, 64, kernel_size=(7, 1, 1), stride=(2, 1, 1), padding=(3, 0, 0), bias=False)
          (1): BatchNorm3d(64, eps=0.001, momentum=0.001, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
      )
      (1): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
      (2): Conv3dNormActivation(
        (0): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=0.001, momentum=0.001, affine=True, track_running_stats=True)
        (2): ReLU(inplace=

In [9]:
activities = [ProfilerActivity.CPU]
if torch.cuda.is_available():
    device = "cuda"
    activities += [ProfilerActivity.CUDA]
elif torch.xpu.is_available():
    device = "xpu"
    activities += [ProfilerActivity.XPU]
else:
    print(
        "Neither CUDA nor XPU devices are available to demonstrate profiling on acceleration devices"
    )
    import sys

    sys.exit(0)

sort_by_keyword = device + "_time_total"

model = model.to(device)
inputs = video.to(device)

with profile(activities=activities, record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

print(prof.key_averages().table(sort_by=sort_by_keyword, row_limit=10))

[W1102 16:11:35.186217439 kineto_shim.cpp:415] Adding profiling metadata requires using torch.profiler with Kineto support (USE_KINETO=1)


RuntimeError: Input type (torch.cuda.ByteTensor) and weight type (torch.cuda.FloatTensor) should be the same

In [ ]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))